In [1]:
# 介绍 : 新算法,不加weibull_noise,所有节点加高斯noise,单个样本循环
    #grad计算方法 : grad = ( a*noise*loss )/sigm**2

In [2]:
# Python imports
import numpy as np # Matrix and vector computation package
import matplotlib.pyplot as plt  # Plotting library
# Allow matplotlib to plot inside this notebook
%matplotlib inline
# Set the seed of the numpy random number generator so that the tutorial is reproducable
np.random.seed(seed=1)
from sklearn import datasets, cross_validation, metrics # data and evaluation utils 
from matplotlib.colors import colorConverter, ListedColormap # some plotting functions
import itertools
import collections
from scipy import stats
from scipy.stats import weibull_min
import matplotlib.pyplot as plt
import random
import math

C:\Users\kzw\AppData\Local\conda\conda\envs\python27\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
#给逻辑层的输出的头部插入1
def array_add_data(activ):
    """
    参数activ : 每层的输出(激活)  第一层 : (32L, 20L)
    作用 : 在数组头部插入数值1
    """
    r = []
    ret1 = 0
    for i in range(activ.shape[0]):
        ret1 = np.array(np.append([1],activ[i])) #list
        r.append(ret1)
    r = np.array(r) # 类型转换:list -> array
    return r

#给所有样本头部插入1
def array_add_data2(activ):
    """
        参数activ : 样本
        作用 : 在数组头部插入数值1
    """
    r = []
    ret1 = np.append([1],activ)
    r.append(ret1)
    ret2 = np.array(r)
    return ret2

In [4]:
# Define the non-linear functions used
def logistic(z): 
    return 1 / (1 + np.exp(-z))

# def logistic_deriv(y):  # Derivative of logistic function
#     return np.multiply(y, (1 - y))
    
def softmax(z): 
    return np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)

In [5]:
# Define the layers used in this model
class Layer(object):
    
#     def get_params_iter(self):
#         return []
    
#     def get_params_grad(self, X, output_grad):
#         return []
      
#     def get_params_update(self, X, loss):
#         return []
    
#     def add_noise(self):
#         pass
    
#     def remove_noise(self):
#         pass
    
    def get_output(self, X):
        pass
    
#     def get_input_grad(self, Y, output_grad=None, T=None):
#         pass

In [6]:
class discreteLayer(Layer):
    """The linear layer performs a linear transformation to its input."""
    
    def __init__(self, n_in, n_out):
        """ 
            对隐藏层参数进行初始化
            n_in是输入变量的数量
            n_out是输出变量的数量
        """
        self.W = np.random.randn(n_in, n_out) * 0.1
        self.W[0:1,:] = 0 
    def get_output(self, X):
        # 线性变换
        xw = X.dot(self.W)
        re = xw + self.noise 
        return re

In [8]:
class LogisticLayer(Layer):
    """The logistic layer applies the logistic function to its inputs."""
    
    def get_output(self, X):
        # 在头部插入数值1
        logi = logistic(X)
        Y = array_add_data(logi)
        return Y

In [9]:
class SoftmaxOutputLayer(Layer):
    """ 
        输出层
    """  
    def get_output(self, X):
        r = softmax(X)
        return r
    
    def get_cost(self, A, T):
        re = - np.multiply(T, np.log(A)).sum() / A.shape[0]      
        return re

In [10]:
def forward_step(input_samples,layers,noise1):
    activations = [input_samples] 
    X = input_samples
    for index in range(len(layers)): 
        layer = layers[index]
        layer.noise = noise1[index]
        Y = layer.get_output(X)
        activations.append(Y)  
        X = activations[-1]  
    return activations  

In [11]:
#得到样本

# 在所有样本的头部添加数值1
digits = datasets.load_digits()
d = digits.data # array
# print(data.shape) # (1797L, 64L)
target_list = [] # list
target_data = []
# single_array = np.array([1])
for i in range(len(d)):
    # 取出数据
    a = d[i]
    # 头部添加1 
    a_temp = array_add_data2(a)[0]
    # 添加到总的array中
    target_list.append(a_temp)
target_data = np.array(target_list) 

T = np.zeros((digits.target.shape[0],10))
T[np.arange(len(T)), digits.target] += 1

#有问题 : 这里的划分应该是随机的,和digits.data不对应
X_train, X_test, T_train, T_test = cross_validation.train_test_split(
    target_data, T, test_size=0.4)
X_validation, X_test, T_validation, T_test = cross_validation.train_test_split(
    X_test, T_test, test_size=0.5)

In [12]:
# # 创建minibatches
# X_train = X_train[0:1024,:] #取1024方便计算
# T_train = T_train[0:1024,:]
batch_size = 1#    adjust
# nb_of_batches = X_train.shape[0] / batch_size  # 32批
# # Create batches (X,Y) from the training set
# XT_batches = zip(
#     np.array_split(X_train, nb_of_batches, axis=0),  
#     np.array_split(T_train, nb_of_batches, axis=0)) 

In [13]:
# 获得grad
def get_grad(ac,noise,sigm,loss):
    grad = []
    gra = 0
    for i in range(len(noise)):
        if i%2 == 0 :
            a = ac[i]
            noi = noise[i]
            do  = a.T.dot(noi)
            gra = ( do * loss ) / (sigm**2)  
            grad.append(gra)
    return grad

In [14]:
# 定义模型
hidden_neurons_1 = 20  # 第一个隐藏层的神经元数目
layers = [] #
# 添加第一个隐层
layers.append(discreteLayer(X_train.shape[1], hidden_neurons_1))  
layers.append(LogisticLayer())
# 添加输出层
layers.append(discreteLayer(21, T_train.shape[1])) 
layers.append(SoftmaxOutputLayer())

(65L, 20L)
(21L, 10L)


In [15]:
#迭代训练 : 单张图片  指定结点 target_data[0]
x1 = target_data[0]
t1 = T[0]
x1 = np.expand_dims(x1, axis=0)
t1 = np.expand_dims(t1, axis=0)

# 开始训练
sigm = 5  #35
max_nb_of_iterations = 50000 # 训练次数
# 选定一个结点
layer_index,node_index,weight_index = 2,1,1  #layer : 1,2
w = []
for iteration in range(max_nb_of_iterations): 
    # 准备好各layer的高斯noise
    noise1 = []
    noise1.append(np.random.randn(1,20)*sigm)
    noise1.append([])
    noise1.append(np.random.randn(1,10)*sigm)
    noise1.append([])

    activations = forward_step(x1,layers,noise1)
    loss = layers[-1].get_cost(activations[-1],t1) 
    grad = get_grad(activations,noise1,sigm,loss) 
    w_grad = grad[layer_index - 1][weight_index][node_index - 1]
    w.append(w_grad)
    # 显示循环次数
    if (iteration + 1) % 10000 == 0:
        print(iteration + 1)
#     print("========================= 一个循环 =========================")
print('均值    : '+ str(np.mean(w)))
print('标准误差 : '+ "{:.15f}".format( np.std(w)/math.sqrt(max_nb_of_iterations) )) 

10000
20000
30000
40000
50000
均值    : -0.8192050492058489
标准误差 : 0.009184831349266


In [500]:
# 验证置信区间
d1 = -0.8118846854684239
eps1 =  0.009158330944701 *2
d2 =  -0.7381045555059148
eps2 =   0.001590605978503 *2
print(d1 + eps1) #大
print(d2 - eps2) #小

-0.793568023579
-0.741285767463


In [422]:
# 第一层随机选四个权重 : 

In [ ]:
新算法(仅高斯noise)     :    layer_index,node_index,weight_index = 1,7,20     
-0.02299495781291697  -0.03208230897517716  0.029691683647328142  0.0005051942812933979  0.015683560345395267   0.010667274704363511
 0.027940004929023     0.027883102498920    0.028175666572146     0.027829473381536      0.027907972439502      0.027906467067519
 
BP(梯度的数值逼近计算grad) : layer_index,node_index,weight_index = 1,7,20     
-0.00013162379408626634 -0.0001316227789260882 -0.00013192415576623873 -0.0001304924823058995 -0.00013046200300625976  -0.00013153744794769384
 0.000000836652731       0.000000810907371      0.000000801110100       0.000000784006068      0.000000810112149        0.000000812706314
    
两算法互比结果 : 都互在置信区间内